<a href="https://colab.research.google.com/github/Ishita216/Ishita216/blob/main/4.missing_values_pipelining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Last amended: 1st June, 2023
# Ref: https://www.kaggle.com/code/alexisbcook/missing-values
# Data Source: https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/data


This [data represents](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/data) the results of a large product testing study. For each product_code you are given a number of product attributes (fixed for the code) as well as a number of measurement values for each individual product, representing various lab testing methods. Each product is used in a simulated real-world environment experiment, and and absorbs a certain amount of fluid (loading) to see whether or not it fails.

Your task is to use the data to predict individual product failures of new codes with their individual lab test results.

# Simple Data pipelining

In [1]:
# 1.0

import pandas as pd

# 1.0.1
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier #CLASSIFICAION PROBLEM
from sklearn.metrics import accuracy_score #predicting accuracy
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline #

# 1.0.2
from pathlib import Path


In [3]:
# 1.1
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
# 1.2 Display from a cell outputs of multiple commands:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [17]:
# 1.2.1 Where is our data:

pathToFolder = "/gdrive/MyDrive"


In [18]:
# 1.2.2 And our file?

path = Path(pathToFolder) / "producttesting.zip"

In [40]:
# 2.1 Load the data
data = pd.read_csv(path)

In [41]:
# 2.2
data.shape  #(26570, 26)
data.tail()


(26570, 26)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
26565,26565,E,158.95,material_7,material_6,6,9,6,16,4,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,26566,E,146.02,material_7,material_6,6,9,10,12,8,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,26567,E,115.62,material_7,material_6,6,9,1,10,1,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0
26568,26568,E,106.38,material_7,material_6,6,9,2,9,4,...,11.392,17.064,17.814,14.928,16.273,15.485,13.624,12.865,730.156,0
26569,26569,E,131.20,material_7,material_6,6,9,6,19,1,...,10.611,15.603,19.703,11.006,15.875,13.366,16.527,17.890,602.354,0


In [42]:
# 2.2.1 Get Target

y = data.pop("failure")

In [43]:
X = data.select_dtypes(exclude = ['object'])
X.pop("id")

0            0
1            1
2            2
3            3
4            4
         ...  
26565    26565
26566    26566
26567    26567
26568    26568
26569    26569
Name: id, Length: 26570, dtype: int64

In [45]:
column_name = list(X.columns)

In [47]:
column_name

['loading',
 'attribute_2',
 'attribute_3',
 'measurement_0',
 'measurement_1',
 'measurement_2',
 'measurement_3',
 'measurement_4',
 'measurement_5',
 'measurement_6',
 'measurement_7',
 'measurement_8',
 'measurement_9',
 'measurement_10',
 'measurement_11',
 'measurement_12',
 'measurement_13',
 'measurement_14',
 'measurement_15',
 'measurement_16',
 'measurement_17']

In [59]:
X.head()

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,80.10,9.0,5.0,7.0,8.0,4.0,18.040,12.518,15.748,19.292,...,20.155,10.672,15.859,17.5940,15.193,15.029,16.040,13.034,14.684,764.100
1,84.89,9.0,5.0,14.0,3.0,3.0,18.213,11.540,17.717,17.893,...,17.889,12.448,17.947,17.9150,11.755,14.732,15.425,14.395,15.631,682.057
2,82.43,9.0,5.0,12.0,1.0,5.0,18.057,11.652,16.738,18.240,...,18.288,12.715,15.607,19.2115,13.798,16.711,18.631,14.094,17.946,663.376
3,101.07,9.0,5.0,13.0,2.0,6.0,17.295,11.188,18.576,18.339,...,19.060,12.471,16.346,18.3770,10.020,15.250,15.562,16.154,17.172,826.282
4,188.06,9.0,5.0,9.0,2.0,8.0,19.346,12.950,16.990,15.746,...,18.093,10.337,17.082,19.9320,12.428,16.182,12.760,13.153,16.412,579.885


In [60]:
X.isnull().sum().sum()

0

In [61]:
si = SimpleImputer(strategy='median')

In [62]:
si.fit(X)

SimpleImputer(strategy='median')

In [63]:
X = si.transform(X)

In [64]:
X = pd.DataFrame(X, columns=column_name)

In [54]:
X.head()

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,80.10,9.0,5.0,7.0,8.0,4.0,18.040,12.518,15.748,19.292,...,20.155,10.672,15.859,17.5940,15.193,15.029,16.040,13.034,14.684,764.100
1,84.89,9.0,5.0,14.0,3.0,3.0,18.213,11.540,17.717,17.893,...,17.889,12.448,17.947,17.9150,11.755,14.732,15.425,14.395,15.631,682.057
2,82.43,9.0,5.0,12.0,1.0,5.0,18.057,11.652,16.738,18.240,...,18.288,12.715,15.607,19.2115,13.798,16.711,18.631,14.094,17.946,663.376
3,101.07,9.0,5.0,13.0,2.0,6.0,17.295,11.188,18.576,18.339,...,19.060,12.471,16.346,18.3770,10.020,15.250,15.562,16.154,17.172,826.282
4,188.06,9.0,5.0,9.0,2.0,8.0,19.346,12.950,16.990,15.746,...,18.093,10.337,17.082,19.9320,12.428,16.182,12.760,13.153,16.412,579.885


In [57]:
X.isnull().sum().sum()

0

In [65]:
rf = RandomForestClassifier()

In [66]:
rf.fit(X, y)

RandomForestClassifier()

In [69]:
s = rf.predict(X)

In [72]:
(s==y).sum()/X.shape[0] #understand the errror and accuracy - 1 is 100% accurate

1.0

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25)

In [77]:
rf.predict(X_train)

array([0, 0, 1, ..., 0, 1, 1])

In [68]:
pipe = make_pipeline(SimpleImputer(strategy = 'mean'), RandomForestClassifier())

In [ ]:
# 3.0 Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
pred = pipe.predict(X_valid)

In [ ]:
accu = accuracy_score(y_valid,pred)
accu   # 79%

0.7922468949943545

# Your Turn

a. Add Standard Scaler and PCA to it  
b. Add cross validation to it

In [ ]:
### ----------------------- Done -----------------